## Analysis of Tweets from Ireland 8th

### Setup

In [1]:
import os
import pyspark
from pyspark.sql import SQLContext

# Add the elasticsearch-hadoop jar
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/jovyan/elasticsearch-hadoop-6.2.2.jar pyspark-shell'
conf = pyspark.SparkConf()

# Point to the master.
conf.setMaster("spark://tweetsets.library.gwu.edu:7101")
import os
import pyspark

# Add the elasticsearch-hadoop jar
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /home/jovyan/elasticsearch-hadoop-6.2.2.jar pyspark-shell'
conf = pyspark.SparkConf()

# Point to the master.
conf.setMaster("spark://tweetsets.library.gwu.edu:7101")
conf.setAppName("ireland-8th-analysis")
conf.set("spark.driver.bindAddress", "0.0.0.0")
# Don't hog all of the cores.
conf.set("spark.cores.max", "3")
# Specify a port for the block manager (which runs as part of the worker). The range 7003-7028 is set 
# to be open in the Spark worker container.
conf.set("spark.blockManager.port", "7003")

# create the context
sc = pyspark.SparkContext(conf=conf)

# Configure for ElasticSearch cluster and index.
es_conf = {"es.nodes": "tweetsets.library.gwu.edu",
           "es.port": "9200",
           "es.resource": "tweets-ba2157/doc",
           "es.read.field.as.array.include": "hashtags,text,urls"}

sqlContext = SQLContext(sc)
tweets_df = sqlContext.read.format("org.elasticsearch.spark.sql").options(**es_conf).load()
tweets_df.createOrReplaceTempView("tweets")

### Count

In [2]:
tweets_df.count()

478303

### Top hashtags

In [3]:
hashtags_df = sqlContext.sql("SELECT hashtag, count(hashtag) from (SELECT explode(hashtags) hashtag FROM tweets) group by hashtag order by count(hashtag) desc")
hashtags_df.show(50, truncate=False)

+-------------------+--------------+
|hashtag            |count(hashtag)|
+-------------------+--------------+
|repealthe8th       |84179         |
|together4yes       |57253         |
|savethe8th         |43656         |
|8thref             |35935         |
|togetherforyes     |18763         |
|lovebothvoteno     |18290         |
|voteyes            |8245          |
|loveboth           |8109          |
|latelate           |5920          |
|men4yes            |5884          |
|votenotoabortion   |5732          |
|latelateshow       |5584          |
|voteno             |5324          |
|prolife            |4596          |
|hometovote         |4342          |
|lifecanvass        |3637          |
|trustwomen         |3483          |
|repeal             |3450          |
|ireland            |3192          |
|mybodymychoice     |2903          |
|私は黙らない0428         |2879          |
|repealth8th        |2614          |
|jointherebellion   |2602          |
|abortion           |2463          |
|

### Top users by all tweet types

In [4]:
screen_name_df = sqlContext.sql("SELECT user_screen_name, count(user_screen_name) from tweets group by user_screen_name order by count(user_screen_name) desc")
screen_name_df.show(50, truncate=False)

+----------------+-----------------------+
|user_screen_name|count(user_screen_name)|
+----------------+-----------------------+
|kaydnan         |3449                   |
|Irishprolifer   |2783                   |
|ShannonBlue     |2433                   |
|mobyrne100      |2375                   |
|EmmaMurphy12150 |2278                   |
|Declan1497      |2225                   |
|MaryOGrady8     |2209                   |
|BernadetteComm1 |2164                   |
|Donnchadh32     |1823                   |
|EamonReilly_com |1727                   |
|Paul71          |1700                   |
|christi85573643 |1627                   |
|rosecaroline9   |1626                   |
|MeathRight2Life |1594                   |
|DLTogether4Yes  |1587                   |
|MandyGall7      |1559                   |
|laurathornton30 |1545                   |
|MaryThorn85     |1528                   |
|babydollirish2  |1474                   |
|ebt51           |1430                   |
|ExposeMedi

### Top users by original tweets only

In [5]:
screen_name_orig_df = sqlContext.sql("SELECT user_screen_name, count(user_screen_name) from tweets where tweet_type='original' group by user_screen_name order by count(user_screen_name) desc")
screen_name_orig_df.show(50, truncate=False)

+----------------+-----------------------+
|user_screen_name|count(user_screen_name)|
+----------------+-----------------------+
|ebt51           |626                    |
|loveboth8       |546                    |
|Savethe8thInfo  |516                    |
|testisfidelis   |417                    |
|Together4yes    |287                    |
|Thebfromtuam    |263                    |
|LoveLifeLove8th |246                    |
|AllPassingThing |218                    |
|lifeinstitute   |209                    |
|JanetOS_        |183                    |
|MarieAFlaherty  |148                    |
|InHerIrishShoes |145                    |
|PacifistIreland |143                    |
|repeal8thfunds  |142                    |
|RosForChoice    |142                    |
|tvcritics       |135                    |
|StopRepealHate  |125                    |
|MandyGall7      |122                    |
|TarynDeVere     |122                    |
|paddyearly      |110                    |
|AnnieKatel

### Top URLs

In [6]:
urls_df = sqlContext.sql("SELECT url, count(url) from (SELECT explode(urls) url FROM tweets) where not url like 'http://twitter.com%' group by url order by count(url) desc")
urls_df.show(50, truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|url                                                                                                                                                              |count(url)|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+
|http://checktheregister.ie                                                                                                                                       |569       |
|http://undecided8.org                                                                                                                                            |314       |
|http://www.irishtimes.com/opinion/anti-abortion-posters-fail-to-take-account-of-life-1.3470187                              

### Top timezones

In [7]:
tz_df = sqlContext.sql("SELECT user_time_zone, count(user_time_zone) FROM tweets group by user_time_zone order by count(user_time_zone) desc")
tz_df.show(10, truncate=False)

+--------------------------+---------------------+
|user_time_zone            |count(user_time_zone)|
+--------------------------+---------------------+
|Dublin                    |113521               |
|Pacific Time (US & Canada)|37240                |
|London                    |30191                |
|Amsterdam                 |19888                |
|Casablanca                |15636                |
|Europe/Dublin             |8694                 |
|Eastern Time (US & Canada)|7565                 |
|Hawaii                    |4303                 |
|Central Time (US & Canada)|3470                 |
|Europe/London             |1890                 |
+--------------------------+---------------------+
only showing top 10 rows



### Top user languages

In [8]:
lang_df = sqlContext.sql("SELECT user_language, count(user_language) FROM tweets group by user_language order by count(user_language) desc")
lang_df.show(10, truncate=False)

+-------------+--------------------+
|user_language|count(user_language)|
+-------------+--------------------+
|en           |441308              |
|en-gb        |19521               |
|ja           |4473                |
|en-GB        |4193                |
|es           |2195                |
|fr           |1582                |
|ga           |992                 |
|de           |878                 |
|it           |812                 |
|ru           |517                 |
+-------------+--------------------+
only showing top 10 rows



### Top retweets

In [9]:
rt_df = sqlContext.sql("SELECT CONCAT('https://twitter.com/', retweeted_quoted_screen_name, '/status/', retweet_quoted_status_id), count(retweet_quoted_status_id) FROM tweets group by retweet_quoted_status_id, retweeted_quoted_screen_name order by count(retweet_quoted_status_id) desc")
rt_df.show(25, truncate=False)

+----------------------------------------------------------------------------------------------+-------------------------------+
|concat(https://twitter.com/, retweeted_quoted_screen_name, /status/, retweet_quoted_status_id)|count(retweet_quoted_status_id)|
+----------------------------------------------------------------------------------------------+-------------------------------+
|https://twitter.com/Together4yes/status/993174221265174529                                    |4549                           |
|https://twitter.com/markohalloran/status/987318846980751360                                   |2251                           |
|https://twitter.com/campaignforleo/status/987646457045020672                                  |1958                           |
|https://twitter.com/amyhuberman/status/987303602514530304                                     |1614                           |
|https://twitter.com/Sarah_Hyland/status/993520504052092928                                    |1

### Top trigrams (combinations of 3 words)

In [11]:
from pyspark.ml.feature import RegexTokenizer, NGram, StopWordsRemover
from pyspark.sql.functions import sort_array, udf, explode
from pyspark.sql.types import ArrayType, StringType


# Text (using distinct)
text_df = tweets_df.select(explode("text").alias("text")).distinct()

# Tokenize
tokenizer = RegexTokenizer(pattern="([:\.!?,]|'s|’s)*\\s+[‘]*", inputCol="text", outputCol="words")
tokenized_df = tokenizer.transform(text_df)

# Stopwords
stop_words = StopWordsRemover.loadDefaultStopWords('english')
stop_words.extend(['rt', ' ', '-', '&amp;', 'it’s', '', 'may', 'see', 'want', 'i’m', 'us', 'make', "we've", "you're", "you've", "don't", "i’ve", 'it', 'they’re', 'don’t', 'lets', 'add'])
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words", stopWords=stop_words)
filtered_df = remover.transform(tokenized_df)

# Remove hashtags and URLs and dupes
def clean(arr):
    new_arr = set()
    for item in arr:
        add_to_arr = True
        for startswith in ('#', 'http'):
            if item.startswith(startswith):
                add_to_arr = False
        if add_to_arr:
            new_arr.add(item)
    return list(new_arr)

clean_udf = udf(lambda arr: clean(arr), ArrayType(StringType()))
clean_df = filtered_df.withColumn("clean_words", clean_udf(filtered_df.filtered_words))

# Sort the words
sorted_df = clean_df.select(sort_array('clean_words').alias('sorted_words'))

ngram = NGram(n=3, inputCol="sorted_words", outputCol="ngrams")
ngram_df = ngram.transform(sorted_df).select(explode('ngrams').alias('ngrams'))
ngram_df.groupBy('ngrams').count().orderBy('count', ascending=False).show(50, truncate=False)


+----------------------------------------------+-----+
|ngrams                                        |count|
+----------------------------------------------+-----+
|vote women yes                                |362  |
|voting women yes                              |243  |
|help please support                           |122  |
|vote voting yes                               |117  |
|8th abortion amendment                        |104  |
|@josephamadigan @simonharristd @together4yes  |99   |
|care change compassion                        |88   |
|@conmurphysport @gordonwdarcy @kevinmcgahern  |85   |
|@andyleeboxing @conmurphysport @gordonwdarcy  |83   |
|irish referendum urged                        |78   |
|@gordonwdarcy @kevinmcgahern @richiesadlier   |75   |
|registered sure vote                          |74   |
|please privilege right                        |74   |
|perfect please privilege                      |73   |
|life perfect please                           |72   |
|@simonhar